## 정밀도와 재현율 Precision and Recall


- 정밀도와 재현율은 Positive 데이터셋의 예측 성능에 초점을 맞춘 평가 지표


##### 정밀도 = TP / (FP + TP) 

* 예측을 Positive로 한 대상 중에 예측과 실제값이 Positive로 일치한 데이터의 비율

##### 재현율(민감도) = TP / (FN + TP)

* 실제값이 Positive인 대상 중에 예측과 실제값이 Positive로 일치한 데이터의 비율
